In [4]:
import pandas as pd
import os

os.chdir("D:/functional-prediction/")

In [2]:
df_variant = pd.read_csv("output/dataset_review.csv", dtype=str).fillna("")

df_variant["function"] = df_variant["function"].str.lower().str.strip()

df_variant = df_variant[(~df_variant["function"].str.contains("uncertain")) & (~df_variant["function"].str.contains("unknown"))]
print(len(df_variant))
df_variant.to_csv("output/dataset_clean.csv", index=False)

372


In [3]:
df_multi = pd.read_csv("output/dataset_multi_var_review.csv", dtype=str).fillna("")

In [7]:
# 区分snp和indel
df_variant["variant"] = [""] * len(df_variant)

df_all = pd.concat([df_variant, df_multi], axis=0)

type_list = []
for index, row in df_all.iterrows():
    if row["reference_allele"] == "-" or row["variant_allele"] == "-" or "del" in row["variant_allele"] or \
    "del" in row["variant"] or "ins" in row["variant"]:
        type_list.append("indel")
    else:
        type_list.append("snp")
df_all["type"] = type_list

In [8]:
df_indel = df_all[df_all["type"] == "indel"]
print(len(df_indel))
df_indel.to_csv("output/dataset_clean_indel.csv", index=False)

34


In [9]:
df_variant = df_all[df_all["type"] == "snp"]

In [17]:
# 用于人工校验重复的变异
__df = df_variant.groupby(["chr", "variant_start", "reference_allele", "variant_allele"])["haplotype_name"].count().reset_index(name='count')
__df.sort_values(by=["count", "variant_start"], ascending=False)[:40]

,chr,variant_start,reference_allele,variant_allele,count
148,19,38500898,C,G,2
135,19,38496283,C,T,2
124,19,38448712,G,C,2
122,19,38446710,G,A,2
213,6,18130687,T,G,2
362,X,154536019,G,A,2
363,X,154536019,G,T,2
348,X,154535270,A,C,2
349,X,154535270,A,G,2
326,X,154534389,C,G,2


In [16]:
df_variant.to_csv("output/dataset_clean_snp.csv", index=False)

In [18]:
df_variant = pd.read_csv("output/dataset_clean_snp_review.csv", dtype=str).fillna("")

In [19]:
len(df_variant)

377

In [20]:
## 药物基因上的变异数
df_variant.groupby(["gene", "chr"])["haplotype_name"].count().reset_index(name='count')

,gene,chr,count
0,CYP2B6,19,26
1,CYP2C19,10,19
2,CYP2C9,10,33
3,CYP2D6,22,27
4,CYP3A5,7,2
5,DPYD,1,60
6,G6PD,X,148
7,NUDT15,13,1
8,RYR1,19,44
9,SLCO1B1,12,5


In [21]:
## 不同功能分组的变异数
df_variant.groupby(["function"])["haplotype_name"].count().reset_index(name='count')

,function,count
0,decreased function,75
1,increased function,45
2,no function,192
3,normal function,65


In [22]:
## 涉及到的染色体
sorted(set(df_variant["chr"].values))

['1', '10', '12', '13', '19', '2', '22', '6', '7', 'X']

In [23]:
# chr annotation search
# 对位点排序加速查询，n^2 时间复杂度提升至n*log(n)

def write_variant_file(df_chr, chr_path, output_path, chr_num):
    def gen_file(total_str, variant_start, filepath, first=True):
        variant_len = len(variant_start)
        count = 0
        cur_str = "{}	{}".format(chr_num, variant_start[count])
        matched = False

        all_pos_list = ["{}	".format(chr_num) + x for x in variant_start]
        matched_pos_list = []

        with open(chr_path, "r") as f:
            if first:
                total_str += f.readline()

            line = f.readline()
            while line:
                if matched and not line.startswith(cur_str):
                    count += 1
                    if count >= variant_len:
                        break
                    matched = False
                    cur_str = "{}	{}".format(chr_num, variant_start[count])

                if line.startswith(cur_str) and count < variant_len:

                    matched_pos_list.append(cur_str)
                    matched = True
                    total_str += line

                line = f.readline()

        matched_pos_list = list(set(matched_pos_list))
        missed_pos_list = sorted(set(all_pos_list) - set(matched_pos_list))
        return missed_pos_list, total_str


    variant_list = list(df_chr["variant_start"].values)
    variant_str = ""

    missed_pos_list, variant_str = gen_file(variant_str, variant_list, chr_path)
    print(missed_pos_list)

    while len(missed_pos_list) > 0:
        missed_pos_list.pop(0)
        
        if len(missed_pos_list) == 0:
            break
            
        missed_pos_list = [x.split("\t")[-1] for x in missed_pos_list]
        missed_pos_list, variant_str = gen_file(variant_str, missed_pos_list, chr_path, first=False)
        print(missed_pos_list)
        
    with open(output_path, "w") as f:
        f.write(variant_str)
    


In [24]:
# chr1 DPYD
df_chr1 = df_variant[df_variant["chr"] == "1"]
df_chr1 = df_chr1.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr1"
output_file = "output/chr1_variant.tsv"

# write_variant_file(df_chr1, path, output_file, "1")

In [25]:
# chr10 CYP2C19 CYP2C9
df_chr10 = df_variant[df_variant["chr"] == "10"]
df_chr10 = df_chr10.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr10"
output_file = "output/chr10_variant.tsv"

# write_variant_file(df_chr10, path, output_file, "10")

In [26]:
# chr19 RYR1
df_chr19 = df_variant[df_variant["chr"] == "19"]
df_chr19 = df_chr19.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr19"
output_file = "output/chr19_variant.tsv"

# write_variant_file(df_chr19, path, output_file, "19")

In [27]:
# chr22 CYP2D6
df_chr22 = df_variant[df_variant["chr"] == "22"]
df_chr22 = df_chr22.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr22"
output_file = "output/chr22_variant.tsv"

# write_variant_file(df_chr22, path, output_file, "22")

In [28]:
# chr12 SLCO1B1
df_chr12 = df_variant[df_variant["chr"] == "12"]
df_chr12 = df_chr12.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr12"
output_file = "output/chr12_variant.tsv"

# write_variant_file(df_chr12, path, output_file, "12")

In [29]:
# chr13 NUDT15
df_chr13 = df_variant[df_variant["chr"] == "13"]
df_chr13 = df_chr13.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr13"
output_file = "output/chr13_variant.tsv"

# write_variant_file(df_chr13, path, output_file, "13")

In [30]:
#chr2 UGT1A1
df_chr2 = df_variant[df_variant["chr"] == "2"]
df_chr2 = df_chr2.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr2"
output_file = "output/chr2_variant.tsv"

# write_variant_file(df_chr2, path, output_file, "2")

In [31]:
#chr6 TPMT
df_chr6 = df_variant[df_variant["chr"] == "6"]
df_chr6 = df_chr6.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr6"
output_file = "output/chr6_variant.tsv"

# write_variant_file(df_chr6, path, output_file, "6")

In [48]:
#chr7 CYP3A5
df_chr7 = df_variant[df_variant["chr"] == "7"]
df_chr7 = df_chr7.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chr7"
output_file = "output/chr7_variant.tsv"

# write_variant_file(df_chr7, path, output_file, "7")

In [32]:
#chrX D6PD
df_chrX = df_variant[df_variant["chr"] == "X"]
df_chrX = df_chrX.sort_values(by=["variant_start"])
path = "d:/download/dbNSFP4.2a_variant.chrX"
output_file = "output/chrX_variant.tsv"

write_variant_file(df_chrX, path, output_file, "X")

['X\t154532278', 'X\t154532279', 'X\t154532389', 'X\t154532390', 'X\t154532392', 'X\t154532403', 'X\t154532408', 'X\t154532411', 'X\t154532432', 'X\t154532434', 'X\t154532458', 'X\t154532459', 'X\t154532570', 'X\t154532608', 'X\t154532623', 'X\t154532625', 'X\t154532626', 'X\t154532628', 'X\t154532629', 'X\t154532634', 'X\t154532639', 'X\t154532649', 'X\t154532661', 'X\t154532662', 'X\t154532667', 'X\t154532674', 'X\t154532676', 'X\t154532677', 'X\t154532679', 'X\t154532688', 'X\t154532692', 'X\t154532694', 'X\t154532695', 'X\t154532698', 'X\t154532699', 'X\t154532700', 'X\t154532701', 'X\t154532713', 'X\t154532715', 'X\t154532716', 'X\t154532722', 'X\t154532758', 'X\t154532765', 'X\t154532772', 'X\t154532773', 'X\t154532797', 'X\t154532802', 'X\t154532945', 'X\t154532969', 'X\t154532987', 'X\t154532989', 'X\t154532990', 'X\t154533004', 'X\t154533016', 'X\t154533029', 'X\t154533031', 'X\t154533044', 'X\t154533064', 'X\t154533072', 'X\t154533077', 'X\t154533083', 'X\t154533122', 'X\t154

In [5]:
df_var = pd.read_csv("output/chr6_variant.tsv", sep="\t", dtype=str).fillna("")

In [6]:
def generate_variant_annotation(df_variant, df_annotation, remain_column):
    df_anno = pd.DataFrame(columns=remain_column)
    exist_list = []
    for index, row in df_variant.iterrows():
        df_annotation_new = df_annotation[
            (df_annotation["pos(1-based)"] == row["variant_start"]) &
            (df_annotation["ref"] == row["reference_allele"]) &
            (df_annotation["alt"] == row["variant_allele"])
        ]
        
        df_annotation_new = df_annotation_new[remain_column]
        df_annotation_new.index = range(len(df_annotation_new))
        if len(df_annotation_new) == 1:
            df_anno = pd.concat([df_anno, df_annotation_new], axis=0)
            exist_list.append(True)
        elif len(df_annotation_new) == 0:
            df_anno = pd.concat([df_anno, pd.DataFrame([[""] * len(remain_column)], columns=remain_column)], axis=0)
            exist_list.append(False)
        else:
            df_anno = pd.concat([df_anno, df_annotation_new.iloc[:1]], axis=0)
            exist_list.append(True)
            
    df_anno.index = range(len(df_anno))
    df_variant.index = range(len(df_variant))
    df_variant = pd.concat([df_variant, df_anno], axis=1)
    df_variant["exist"] = exist_list
    df_variant = df_variant[df_variant["exist"] == True]
    df_variant.pop("exist")
    return df_variant
    

In [7]:
score_rank_list = [
    "DEOGEN2_score", "MPC_score", "integrated_fitCons_score", "FATHMM_score", "LIST-S2_score",
    "PrimateAI_score", "PROVEAN_score", "MutationAssessor_score", "BayesDel_addAF_score", 
    "MetaLR_score", "MetaSVM_score", "VEST4_score", "BayesDel_noAF_score", "GERP++_RS", 
    "GenoCanyon_score", "ClinPred_score", "MVP_score", "REVEL_score", "SIFT_score", "Polyphen2_HDIV_score",
    "Polyphen2_HVAR_score", "phastCons17way_primate", "bStatistic", "DANN_score", "CADD_raw"
]

In [8]:
score_column = list(filter(lambda x: "score" in x or "primate" in x or "prob" in x, df_var.columns))

In [9]:
selected_column = list(set(score_rank_list).union(set(score_column)))

In [14]:
pd.DataFrame({"column": selected_column}).to_csv("output/selected_column.csv", index=False)

In [39]:
# chr1 feature
df_chr1_anno = pd.read_csv("output/chr1_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr1_feat = generate_variant_annotation(df_chr1, df_chr1_anno, selected_column)

In [40]:
# chr10 feature
df_chr10_anno = pd.read_csv("output/chr10_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr10_feat = generate_variant_annotation(df_chr10, df_chr10_anno, selected_column)

In [41]:
# chr12 feature
df_chr12_anno = pd.read_csv("output/chr12_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr12_feat = generate_variant_annotation(df_chr12, df_chr12_anno, selected_column)

In [42]:
# chr13 feature
df_chr13_anno = pd.read_csv("output/chr13_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr13_feat = generate_variant_annotation(df_chr13, df_chr13_anno, selected_column)

In [43]:
# chr19 feature
df_chr19_anno = pd.read_csv("output/chr19_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr19_feat = generate_variant_annotation(df_chr19, df_chr19_anno, selected_column)

In [44]:
# chr2 feature
df_chr2_anno = pd.read_csv("output/chr2_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr2_feat = generate_variant_annotation(df_chr2, df_chr2_anno, selected_column)

In [45]:
# chr22 feature
df_chr22_anno = pd.read_csv("output/chr22_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr22_feat = generate_variant_annotation(df_chr22, df_chr22_anno, selected_column)

In [46]:
# chr6 feature
df_chr6_anno = pd.read_csv("output/chr6_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr6_feat = generate_variant_annotation(df_chr6, df_chr6_anno, selected_column)

In [49]:
# chr7 feature
# no result
df_chr7_anno = pd.read_csv("output/chr7_variant.tsv", sep="\t", dtype=str).fillna("")
df_chr7_feat = generate_variant_annotation(df_chr7, df_chr7_anno, selected_column)

In [50]:
# chrX feature
df_chrX_anno = pd.read_csv("output/chrX_variant.tsv", sep="\t", dtype=str).fillna("")
df_chrX_feat = generate_variant_annotation(df_chrX, df_chrX_anno, selected_column)

In [54]:
df_feat = pd.concat([df_chr1_feat, df_chr10_feat, df_chr12_feat, df_chr13_feat, 
                     df_chr19_feat, df_chr2_feat, df_chr22_feat, df_chr6_feat, 
                     df_chr7_feat, df_chrX_feat], axis=0)

In [55]:
_df1 = df_feat.groupby(["gene", "chr"])["haplotype_name"].count().reset_index(name='gene_count')

In [56]:
_df2 = df_feat.groupby(["gene", "function"])["haplotype_name"].count().reset_index(name='count')
_df2 = pd.merge(_df2, _df1, how='left', on=['gene'])
_df2 = _df2.assign(percentage=_df2.apply(lambda row: "{}%".format(round(row['count'] / row['gene_count'] * 100, 2)), axis=1))
_df2.pop("chr")
_df2.pop("gene_count")
_df2

,gene,function,count,percentage
0,CYP2B6,decreased function,4,16.0%
1,CYP2B6,increased function,1,4.0%
2,CYP2B6,no function,12,48.0%
3,CYP2B6,normal function,8,32.0%
4,CYP2C19,decreased function,6,33.33%
5,CYP2C19,no function,8,44.44%
6,CYP2C19,normal function,4,22.22%
7,CYP2C9,decreased function,21,63.64%
8,CYP2C9,no function,11,33.33%
9,CYP2C9,normal function,1,3.03%


In [57]:
_df2 = df_feat.groupby(["function"])["haplotype_name"].count().reset_index(name='count')
sum_count = sum(_df2["count"].values)
_df2 = _df2.assign(percentage=_df2.apply(lambda row: "{}%".format(round(row['count'] / sum_count * 100, 2)), axis=1))
_df2

,function,count,percentage
0,decreased function,74,20.22%
1,increased function,43,11.75%
2,no function,187,51.09%
3,normal function,62,16.94%


In [58]:
len(df_feat)

366

In [59]:
df_feat.to_csv("output/variant_feat.csv", index=False)

In [60]:
len(selected_column)

86

In [61]:
df_lof = pd.read_csv("data/LoFtool_scores.txt", sep="\t")
lof_dict = dict(zip(
    df_lof["Gene"].values,
    df_lof["LoFtool_percentile"].values
))
gene_lof_list = [str(lof_dict.get(x, "")) for x in df_feat["gene"].values]

df_feat["LoFtool"] = gene_lof_list

In [62]:
chr_list = df_feat["chr"].values
variant_start_list = df_feat["variant_start"].values
reference_allele_list = df_feat["reference_allele"].values
variant_allele_list = df_feat["variant_allele"].values

feat_variant = [
    "{}:g.{}{}>{}".format(x[0], x[1], x[2], x[3]) for x in 
     zip(chr_list, variant_start_list, reference_allele_list, variant_allele_list)
]
df_feat["variant"] = feat_variant

In [63]:
# clean_feat

def is_float(value):
    try:
        float(value)
        return True
    except:
        return False


for index, row in df_feat.iterrows():
    for col in df_feat.columns:
        if row[col] == '.':
            row[col] = ""

        if ";" in row[col]:
            val_str = row[col].strip().split(";")
            val_str = list(filter(lambda x: x!= "" and x!= "." and is_float(x), val_str))
            if len(val_str) > 0:
                row[col] = str(min([float(x) for x in val_str]))
            else:
                row[col] = ""


df_feat.to_csv("output/variant_feat_clean.csv", index=False)

In [64]:
import requests
import pandas as pd
import os
import json

os.chdir("D:/functional-prediction/")

In [65]:
df_snp = pd.read_csv("output/dataset_clean_snp_review.csv", dtype=str).fillna("")

chr_list = df_snp["chr"].values
variant_start_list = df_snp["variant_start"].values
reference_allele_list = df_snp["reference_allele"].values
variant_allele_list = df_snp["variant_allele"].values

notations = [
    "{}:g.{}{}>{}".format(x[0], x[1], x[2], x[3]) for x in 
     zip(chr_list, variant_start_list, reference_allele_list, variant_allele_list)
]

In [66]:
# VEP data source
# https://rest.ensembl.org/documentation/info/vep_hgvs_get

path = "https://rest.ensembl.org/vep/human/hgvs/"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

batch = 100

notation_list = []

for i in range(int(len(notations) / batch) + 1):
    print(i)
    r = requests.post(
        path,
        headers=headers,
        json={ 
            "hgvs_notations" : notations[i * batch : (i + 1) * batch]
        },
        timeout=30
    )
    resp_list = json.loads(r.text)
    notation_list.extend(resp_list)

In [67]:
notation_dict = {x["id"]: x for x in notation_list}

In [69]:
with open("output/notation_dict.json", "w") as f:
    json.dump(notation_dict, f)

In [70]:
from collections import Counter
Counter([x["most_severe_consequence"] for x in notation_list]).most_common()

[('missense_variant', 345),
 ('stop_gained', 15),
 ('synonymous_variant', 6),
 ('splice_acceptor_variant', 5),
 ('start_lost', 3),
 ('splice_donor_variant', 2),
 ('upstream_gene_variant', 1)]

In [71]:
notation_list[1].keys()

dict_keys(['id', 'colocated_variants', 'allele_string', 'assembly_name', 'seq_region_name', 'transcript_consequences', 'most_severe_consequence', 'start', 'end', 'input', 'strand'])

In [72]:
# 通过变异位点获取基因名
gene_symbol_list = []
for notat in notation_list:
    gene_symbol_list.append(list(set([x.get("gene_symbol", "") for x in notat["transcript_consequences"]]) - {""})[0])

In [73]:
df_lof = pd.read_csv("data/LoFtool_scores.txt", sep="\t")
lof_dict = dict(zip(
    df_lof["Gene"].values,
    df_lof["LoFtool_percentile"].values
))
gene_lof_list = [lof_dict.get(x, "") for x in gene_symbol_list]

In [74]:
notat_dict = {}
all_notations = []
for x in notation_list:
    ct_list = []
    for y in x["transcript_consequences"]:
        ct_list.extend(y["consequence_terms"])
    ct_list = list(set(ct_list))
    all_notations.extend(ct_list)
    notat_dict[x["id"]] = {"consequence_terms": ct_list, "most_severe_consequence": x["most_severe_consequence"]}

In [75]:
not_df_dict = {}

not_df_dict["variant"] = []

for key in set(all_notations):
    not_df_dict[key] = []
    
for key, value in notat_dict.items():
    exist_cons = value["consequence_terms"]
    not_exist_cons = list(set(all_notations) - set(exist_cons))
    for ec in exist_cons:
        not_df_dict[ec].append(1)
    for ec in not_exist_cons:
        not_df_dict[ec].append(0)
    not_df_dict["variant"].append(key)
        

In [76]:
df_notation = pd.DataFrame(not_df_dict)
df_notation.to_csv("output/snp_variant_type.csv", index=False)

In [77]:
for col in list(set(list(df_notation.columns)) - set("variant")):
    if col == "variant":
        continue
    print("{}: {}".format(col, sum(df_notation[col].values)))

upstream_gene_variant: 143
synonymous_variant: 27
NMD_transcript_variant: 273
non_coding_transcript_variant: 38
splice_donor_variant: 2
splice_acceptor_variant: 5
5_prime_UTR_variant: 4
incomplete_terminal_codon_variant: 1
non_coding_transcript_exon_variant: 214
splice_region_variant: 14
stop_gained: 15
splice_polypyrimidine_tract_variant: 1
start_lost: 3
intron_variant: 72
missense_variant: 348
downstream_gene_variant: 175
coding_sequence_variant: 1
3_prime_UTR_variant: 178


In [78]:
pd.merge(df_feat, df_notation, how="left", on=["variant"]).to_csv("output/variant_feat_clean.csv", index=False)

In [79]:
# 添加APF score
# https://www.nature.com/articles/s41397-018-0044-2

import yaml

In [80]:
with open("threshold.yml", "r") as f:
    thres_dict = yaml.safe_load(f)

In [81]:
df_feat = pd.read_csv("output/variant_feat_clean.csv")

In [82]:
apf_score_list = []
for index, row in df_feat.iterrows():
    cur_list = []
    for key, value in thres_dict.items():
        score = row[key]
        if not score:
            continue
        
        if (value["type"] == "lt" and score < value["threshold"]) or \
           (value["type"] == "gt" and score > value["threshold"]):
            cur_list.append(1)
        else:
            cur_list.append(0)
        
    if len(cur_list) == 0:
        apf_score_list.append(None)
        
    else:
        apf_score_list.append(round(sum(cur_list) / len(thres_dict.keys()), 3))
        

In [83]:
df_feat["APF_score"] = apf_score_list

In [84]:
df_feat

,gene,haplotype_name,chr,variant_start,reference_allele,variant_allele,function,variant,type,Polyphen2_HDIV_score,...,splice_region_variant,stop_gained,splice_polypyrimidine_tract_variant,start_lost,intron_variant,missense_variant,downstream_gene_variant,coding_sequence_variant,3_prime_UTR_variant,APF_score
0,DPYD,rs148799944,1,97078993,C,G,normal function,1:g.97078993C>G,snp,0.000,...,0,0,0,0,0,1,0,0,0,0.167
1,DPYD,rs140114515,1,97079005,C,T,normal function,1:g.97079005C>T,snp,0.000,...,0,0,0,0,0,1,0,0,0,0.000
2,DPYD,rs139459586,1,97079076,A,C,normal function,1:g.97079076A>C,snp,0.954,...,0,0,0,0,0,1,0,0,0,0.667
3,DPYD,rs202144771,1,97079077,G,A,normal function,1:g.97079077G>A,snp,0.021,...,0,0,0,0,0,1,0,0,0,0.167
4,DPYD,rs72547601,1,97079121,T,C,no function,1:g.97079121T>C,snp,1.000,...,0,0,0,0,0,1,0,0,0,0.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,G6PD,G6PD Orissa,X,154536168,G,C,decreased function,X:g.154536168G>C,snp,0.998,...,0,0,0,0,0,1,0,0,0,0.500
362,G6PD,G6PD Rignano,X,154536169,C,T,decreased function,X:g.154536169C>T,snp,1.000,...,0,1,0,0,0,1,0,0,0,0.333
363,G6PD,G6PD Gaohe,X,154546061,T,C,decreased function,X:g.154546061T>C,snp,1.000,...,0,0,0,0,1,1,0,0,0,0.833
364,G6PD,G6PD Lages,X,154546116,C,T,decreased function,X:g.154546116C>T,snp,0.999,...,0,0,0,0,1,1,0,0,0,0.833


In [85]:
df_feat.to_csv("output/variant_feat_clean.csv", index=False)

In [88]:
with open("gene_range.yml", "r") as f:
    gene_range_dict = yaml.safe_load(f)

In [89]:
gene_range_dict

{'CYP2B6': {'chormosome': 19, 'start': 40991282, 'end': 41018398},
 'CYP2C19': {'chormosome': 10, 'start': 94762681, 'end': 94855547},
 'CYP2C9': {'chormosome': 10, 'start': 94938658, 'end': 94990091},
 'CYP2D6': {'chormosome': 22, 'start': 42126499, 'end': 42130810},
 'DPYD': {'chormosome': 1, 'start': 97077743, 'end': 97921059},
 'NUDT15': {'chormosome': 13, 'start': 48037726, 'end': 48052755},
 'RYR1': {'chormosome': 19, 'start': 38433691, 'end': 38587564},
 'SLCO1B1': {'chormosome': 12, 'start': 21131194, 'end': 21239796},
 'TPMT': {'chormosome': 6, 'start': 18128311, 'end': 18155169},
 'UGT1A1': {'chormosome': 2, 'start': 233760270, 'end': 233773300}}

In [92]:
df_indel = pd.read_csv("output/dataset_clean_indel.csv", dtype=str).fillna("")

In [94]:
df_indel.groupby(["type", "function"])["haplotype_name"].count().reset_index(name="count")

,type,function,count
0,indel,decreased function,1
1,indel,no function,33
